In [1]:
%serialconnect

Connecting to Serial /dev/ttyUSB0 baud=115200   ** connected
normal repl mode not detected Ready.

In [36]:
%sendtofile --source pca9685servo.py

Sent 48 lines (1457 bytes).

In [5]:
from pca9685servo import setangspeed 

In [6]:
a0 = 138-8  # servo2 arm vertical
b0 = 118+5  # servo1 pencil horizontal
setangspeed([b0,a0,180])


.

In [96]:
%%writefile servopencil.py

from pca9685servo import setangspeed 
import math, time

class servopencil:
    def __init__(self, h, s, p, e, a0, b0, r0, t0):
        self.h = h  # height servo2 axle
        self.s = s  # servo1 to 2 axle dist
        self.p = p  # pencil length
        self.e = e  # servo3 to pencil centre dist

        self.hps1 = -h**2 + (p+s)**2
        self.hps2 = h**2 - (p-s)**2
        self.hps3 = (h-s)**2 - p**2

        self.minx = math.sqrt(p**2 - (h+s)**2)
        self.maxx = math.sqrt(self.hps1)
        print("xrange", self.minx, self.maxx)
        
        self.a0 = a0  # vertical
        self.b0 = b0  # horizontal
        self.r0 = r0  # zero rad
        self.t0 = t0  # zero theta

        self.xp = 0
        self.yp = 0
        setangspeed([b0,a0,t0])

    def solveab(self, x):
        x2 = x**2
        a = 2*math.atan((2*self.s*x - math.sqrt((self.hps1 - x2)*(self.hps2 + x2)))/(self.hps3 + x2))
        b = math.acos((-self.s*math.sin(a) + x)/self.p)
        return math.degrees(a), math.degrees(b)

    def xy(self, x, y, bplus=0):
        rlsq = (self.e+x)**2 + (self.r0+y)**2
        r = math.sqrt(rlsq - self.e**2)
        t = math.degrees(math.atan(r/self.e) - math.atan((self.r0+y)/(self.e+x)))
        a, b = self.solveab(r)
        setangspeed([self.b0-(b-a)+bplus, self.a0-a, self.t0-t], 0)

    def goxy(self, x, y, bplus=0, feedrate=100):
        n = max(abs(x-self.xp), abs(y-self.yp), 1)
        for i in range(0, n+1):
            l = i/n
            self.xy(self.xp*(1-l) + x*l, self.yp*(1-l) + y*l, bplus)
            time.sleep(1/feedrate)
        self.xp = x
        self.yp = y


Writing servopencil.py



In [7]:
a0 = 138-8  # servo2 arm vertical
b0 = 118+5  # servo1 pencil horizontal
r0, t0 = 120, 90
sp = servopencil(43.8, 67.4, 150, 39.7, a0, b0, r0, t0)


Traceback (most recent call last):
  File "<stdin>", line 4, in <module>
NameError: name 'servopencil' is not defined


In [85]:
setangspeed([b0,a0,90])


In [90]:
sp.xyf(90,20,0, 200)


In [92]:
# do a spiral
for cr in range(5, 35, 5):
    for i in range(160):
        sp.xyf(20+math.sin(i*0.2)*cr, math.cos(i*0.2)*cr, (15 if i==0 else 0), 200)
sp.xyf(0,0,15)


.......

In [ ]:
# now commence with some backlash experiments